# SDR operation HSP Summary
## Preprocess Procedure
### Read data from raw file

In [ ]:
import pandas as pd

raw_file = './data/SDR_Operation_HSP_20200830.xlsx'

key_col = '序号'

pre_op_df = pd.read_excel(raw_file, sheet_name='Pre-Operation', index_col=key_col)
pst_1m_df = pd.read_excel(raw_file, sheet_name='Post-Operation 1month', index_col=key_col)
pst_3m_df = pd.read_excel(raw_file, sheet_name='Post-Operation 3month', index_col=key_col)
pst_6m_df = pd.read_excel(raw_file, sheet_name='Post-Operation 6month', index_col=key_col)
pst_12m_df = pd.read_excel(raw_file, sheet_name='Post-Operation 12month', index_col=key_col)

In [ ]:
raw_file = './data/SDR_Operation_HSP_20200830.xlsx'

key_col = '序号'

pre_op_df = pd.read_excel(raw_file, sheet_name='Pre-Operation', index_col=key_col)
pst_1m_df = pd.read_excel(raw_file, sheet_name='Post-Operation 1month', index_col=key_col)
pst_3m_df = pd.read_excel(raw_file, sheet_name='Post-Operation 3month', index_col=key_col)
pst_6m_df = pd.read_excel(raw_file, sheet_name='Post-Operation 6month', index_col=key_col)
pst_12m_df = pd.read_excel(raw_file, sheet_name='Post-Operation 12month', index_col=key_col)

In [ ]:
base_columns = ['姓名', '性别', '年龄', '临床诊断']
metrics_columns = list(set(pre_op_df.columns) - set(base_columns))

patient_base_df = pre_op_df[base_columns]
pre_op_df = pre_op_df[metrics_columns]
pst_1m_df = pst_1m_df[metrics_columns]
pst_3m_df = pst_3m_df[metrics_columns]
pst_6m_df = pst_6m_df[metrics_columns]
pst_12m_df = pst_12m_df[metrics_columns]

def add_suffix(origin: list, suffix: str) -> list:
    return ['_'.join([ele, suffix]) for ele in origin]

pre_op_df.columns = add_suffix(pre_op_df.columns.tolist(), 'pre_op')
pst_1m_df.columns = add_suffix(pst_1m_df.columns.tolist(), 'pst_1m')
pst_3m_df.columns = add_suffix(pst_3m_df.columns.tolist(), 'pst_3m')
pst_6m_df.columns = add_suffix(pst_6m_df.columns.tolist(), 'pst_6m')
pst_12m_df.columns = add_suffix(pst_12m_df.columns.tolist(), 'pst_12m')

data = pd.merge(patient_base_df, pre_op_df, how='left', on=key_col)
data = pd.merge(data, pst_1m_df, how='left', on=key_col)
data = pd.merge(data, pst_3m_df, how='left', on=key_col)
data = pd.merge(data, pst_6m_df, how='left', on=key_col)
data = pd.merge(data, pst_12m_df, how='left', on=key_col)

In [ ]:
object_columns = [col for col in data.columns.to_list() if str(data[col].dtype) == 'object']
print(object_columns)

data.describe()

- fix log

-- trim column 'P, Q, R, S' in sheet 'Post-Operation 3month', then update the same column titles for all other sheet

-- change value '1+' to '1.5' in sheet 'Post-Operation 12month'

### fillna
- fill na value with mean value of columns

In [ ]:
na_columns = data.columns[data.isna().any()].to_list()
na_value = {col: val for col, val in zip(na_columns, data[na_columns].mean())}
data = data.fillna(value=na_value)

In [ ]:
data.describe()

- export to file

In [ ]:
data.to_excel('./data/merged_data.xlsx')

## EDA

In [1]:
import pandas as pd
import pandas_profiling as profiling
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_excel('./data/merged_data.xlsx')

In [3]:
profile = profiling.ProfileReport(df)

In [5]:
profile.to_file('./temp/sdr_hsp_profile.html')

Signature: profile.to_html() -> str
Docstring:
Generate and return complete template as lengthy string
    for using with frameworks.

Returns:
    Profiling report html including wrapper.
File:      d:\miniconda3\lib\site-packages\pandas_profiling\profile_report.py
Type:      method
